In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Hit:13 http:

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-11 00:48:44--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-09-11 00:48:44 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReviewETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
auto_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
auto_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [5]:
auto_df.count()

3514942

In [6]:
auto_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
from pyspark.sql.functions import to_date
review_list = ['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']
review_id_df = auto_df.select([i for i in review_list])
review_id_df = review_id_df.withColumnRenamed('review_date', 'review_date_need_change')
review_id_df = review_id_df.select('*', to_date('review_date_need_change', 'yyyy-mm-dd').alias('review_date'))
review_id_df = review_id_df.drop('review_date_need_change')
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [8]:
product_df = auto_df.select('product_id', 'product_title')
product_df.printSchema()
product_df.count()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



3514942

In [9]:
product_df = product_df.dropDuplicates(['product_id'])
product_df.count()

762982

In [10]:
test = product_df.groupBy('product_id').count()
test.orderBy(test["count"].desc()).select("*").show()

+----------+-----+
|product_id|count|
+----------+-----+
|B00SN7KX54|    1|
|B000C81ZJA|    1|
|B00026Z3E0|    1|
|B001B5EBUI|    1|
|B004LF54X8|    1|
|B00EAOIHR0|    1|
|B00PYCXH7C|    1|
|B00HHS4LKS|    1|
|B00C4B1F7U|    1|
|B00HW1TLYQ|    1|
|B00C503ZUU|    1|
|B00BTMV4YE|    1|
|B000BZEHW6|    1|
|B0029XM950|    1|
|B00LNGEJEI|    1|
|B003CJPSW6|    1|
|B004BL71TC|    1|
|B00J5EYWAG|    1|
|B00X9IL5RW|    1|
|B00UU1DPRY|    1|
+----------+-----+
only showing top 20 rows



In [11]:
customer_df = auto_df.groupby('customer_id').count().withColumnRenamed('count', 'customer_count')
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28258386|             3|
|   18634862|             3|
|    2072312|             3|
|   33132797|            15|
|   14539137|             1|
|   14552054|             5|
|    9347763|             1|
|   43626894|             2|
|   27314089|            47|
|   36829268|             1|
|   24540309|             2|
|    8501205|             1|
|    4361670|             2|
|   30416085|             2|
|    8306731|             1|
|   11491871|             1|
|   46406103|             1|
|   18417791|             1|
|   51299913|             1|
|   20587872|             4|
+-----------+--------------+
only showing top 20 rows



In [12]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [13]:
vine_df = auto_df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
 # Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [15]:
 # Write DataFrame to table
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [16]:
 # Write DataFrame to table
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [17]:
 # Write DataFrame to table
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [18]:
 # Write DataFrame to table
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)